<a href="https://colab.research.google.com/github/Tiabet/Project_Market/blob/master/Skipgram_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 35.7 MB/s eta 0:00:00


In [4]:
import pandas as pd

stop_words = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()
stop_words[:10]

[['휴'],
 ['아이구'],
 ['아이쿠'],
 ['아이고'],
 ['어'],
 ['나'],
 ['우리'],
 ['저희'],
 ['따라'],
 ['의해']]

In [40]:
import numpy as np
import string
from nltk.corpus import stopwords

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

class word2vec(object):
    def __init__(self):
        self.N = 10
        self.X_train = []
        self.y_train = []
        self.window_size = 1
        self.alpha = 0.001
        self.words = []
        self.word_index = {}

    def initialize(self,V,data):
        self.V = V
        self.W = np.random.uniform(-0.8, 0.8, (self.V, self.N))
        self.W1 = np.random.uniform(-0.8, 0.8, (self.N, self.V))

        self.words = data
        for i in range(len(data)):
            self.word_index[data[i]] = i


    def feed_forward(self,X):
        self.h = np.dot(self.W.T,X).reshape(self.N,1)
        self.u = np.dot(self.W1.T,self.h)
        #print(self.u)
        self.y = softmax(self.u)
        return self.y

    def backpropagate(self,x,t):
        e = self.y - np.asarray(t).reshape(self.V,1)
        # e.shape is V x 1
        dLdW1 = np.dot(self.h,e.T)
        X = np.array(x).reshape(self.V,1)
        dLdW = np.dot(X, np.dot(self.W1,e).T)
        self.W1 = self.W1 - self.alpha*dLdW1
        self.W = self.W - self.alpha*dLdW

    def train(self,epochs):
        for x in range(1,epochs):
            self.loss = 0
            for j in range(len(self.X_train)):
                self.feed_forward(self.X_train[j])
                self.backpropagate(self.X_train[j],self.y_train[j])
                C = 0
                for m in range(self.V):
                    if(self.y_train[j][m]):
                        self.loss += -1*self.u[m][0]
                        C += 1
                self.loss += C*np.log(np.sum(np.exp(self.u)))
            print("epoch ",x, " loss = ",self.loss)
            self.alpha *= 1/( (1+self.alpha*x) )

    def predict(self,word,number_of_predictions):
        if word in self.words:
            index = self.word_index[word]
            X = [0 for i in range(self.V)]
            X[index] = 1
            prediction = self.feed_forward(X)
            output = {}
            for i in range(self.V):
                output[prediction[i][0]] = i

            top_context_words = []
            for k in sorted(output,reverse=True):
                top_context_words.append(self.words[output[k]])
                if(len(top_context_words)>=number_of_predictions):
                    break

            return top_context_words
        else:
            print("Word not found in dictionary")

In [52]:
from konlpy.tag import Okt
import string

def preprocessing(corpus):
    okt = Okt()
    training_data = []
    sentences = corpus.split("-")

    for i in range(len(sentences)):
        sentences[i] = sentences[i].strip()
        words_and_tags = okt.pos(sentences[i])

        filtered_words = [word for word, tag in words_and_tags
                          if tag in ['Adverb', 'Verb', 'Adjective']
                          and word not in stop_words]

        x = [word.strip(string.punctuation) for word in filtered_words]
        x = [word for word in filtered_words if len(x) > 1]
        training_data.append(x)

    return training_data


def prepare_data_for_training(sentences,w2v):
    data = {}
    for sentence in sentences:
        for word in sentence:
            if word not in data:
                data[word] = 1
            else:
                data[word] += 1
    V = len(data)
    data = sorted(list(data.keys()))
    vocab = {}
    for i in range(len(data)):
        vocab[data[i]] = i

    #for i in range(len(words)):
    for sentence in sentences:
        for i in range(len(sentence)):
            center_word = [0 for x in range(V)]
            center_word[vocab[sentence[i]]] = 1
            context = [0 for x in range(V)]

            for j in range(i-w2v.window_size,i+w2v.window_size):
                if i!=j and j>=0 and j<len(sentence):
                    context[vocab[sentence[j]]] += 1
            w2v.X_train.append(center_word)
            w2v.y_train.append(context)
    w2v.initialize(V,data)

    return w2v.X_train,w2v.y_train

In [ ]:
epochs = 1000

training_data = preprocessing(corpus)
w2v = word2vec()

prepare_data_for_training(training_data,w2v)
w2v.train(epochs)

print(w2v.predict("좋아요",10))

epoch  1  loss =  1371.791637051001
epoch  2  loss =  1370.542136534754
epoch  3  loss =  1369.3003470261356
epoch  4  loss =  1368.0673793871215
epoch  5  loss =  1366.8443071136012
epoch  6  loss =  1365.6321607500129
epoch  7  loss =  1364.4319228224065
epoch  8  loss =  1363.2445233489614
epoch  9  loss =  1362.0708359737293
epoch  10  loss =  1360.9116747553094
epoch  11  loss =  1359.767791627982
epoch  12  loss =  1358.6398745389197
epoch  13  loss =  1357.5285462519587
epoch  14  loss =  1356.4343637963339
epoch  15  loss =  1355.3578185282645
epoch  16  loss =  1354.2993367641423
epoch  17  loss =  1353.259280937047
epoch  18  loss =  1352.2379512227478
epoch  19  loss =  1351.2355875778921
epoch  20  loss =  1350.2523721310922
epoch  21  loss =  1349.2884318673148
epoch  22  loss =  1348.3438415470625
epoch  23  loss =  1347.4186268040166
epoch  24  loss =  1346.5127673680047
epoch  25  loss =  1345.6262003641098
epoch  26  loss =  1344.7588236431927
epoch  27  loss =  1343.9

In [50]:
print(w2v.predict("가격",10))

['고장', '놀았는데', '작은', '저렴한듯', '하네요', '던져주니', '크지도', '사주려고', '삑', '뜯어']


In [23]:
import csv

# Replace 'data.tsv' with the path to your TSV file
tsv_file = '/content/drive/MyDrive/Coda_Result/2199261531.tsv'

# Initialize an empty list to store the data from the TSV file
data = []

# Read the TSV file and store its content in the 'data' list
with open(tsv_file, 'r', newline='', encoding='utf-8') as file:
    reader = csv.reader(file, delimiter='\t')
    for row in reader:
        data.append(row)

df = pd.DataFrame(data[1:], columns=data[0])

df

,user_id,score,date,review,is_month,is_repurch
0,sati*****,5,23.08.01.,진짜 귀얍게 잘 갖고 놀아요,True,False
1,kb****,5,23.07.03.,강아지 최애 공입니다. 노랑색을 유난히 좋아하네요,True,False
2,kare*****,5,23.05.24.,노견 푸들이 공놀이를 너무 좋아하는데 이빨이 약해서 탄탄한 공은 잘 못 물더라구요....,False,False
3,cm****,5,23.05.18.,저희 강아지가 진짜 좋아하는 공이라 여러개 샀는데 좋아요,True,False
4,meji****,5,22.11.10.,온것중에 빨강이 골라서 줬는데 환장하구 놀아욤 !!\n막 뜯어버리는 애도 아니고 짱...,False,False
...,...,...,...,...,...,...
63,haha****,5,17.10.28.,잘삿어요 이건 뜯지도 못하고 애들이 잘가지고 노네요 ㅎㅎ\n잘산거같아요ㅎㅎ 견당 하...,False,False
64,haha****,5,17.10.28.,만족합니다,False,False
65,hyun*****,5,17.10.12.,너무너무 좋아하네요 5분만에 삑삑이 고장내고도 계속 가지고 놀아요 ^^,False,False
66,ooy_***,5,17.10.20.,배송도 빠르고 강아지가 좋아해요^^,False,False


In [33]:
# 말뭉치 생성
corpus = "-".join(df['review'].tolist())
corpus

'진짜 귀얍게 잘 갖고 놀아요-강아지 최애 공입니다. 노랑색을 유난히 좋아하네요-노견 푸들이 공놀이를 너무 좋아하는데 이빨이 약해서 탄탄한 공은 잘 못 물더라구요. 그런데 어쩌다 이 공을 추천받아서 써보니 말랑하면서 잘 던져지고 내구성도 좋아서 엄청 오래쓰다가 새로 구입하게됐어요! 4개 구매하니 알아서 색별로 챙겨주셔서 너무 감사합니다~!-저희 강아지가 진짜 좋아하는 공이라 여러개 샀는데 좋아요-온것중에 빨강이 골라서 줬는데 환장하구 놀아욤 !!\n막 뜯어버리는 애도 아니고 짱짱해서 오래 가지고 놀 것 같아 더 좋네요ㅎㅎ-가격만족 배송만족 제품만족-아주 잘 쓰고 있어요-뽁뽁이 소리나는 공을 좋아해서 늘 갖고 놀던 뽁뽁이 장난감이 소리가 안나서 가격도 저렴하고 해서 하나 구매해봤어요! 던져주니 아주 좋아해서 만족합니다! 그리고 라텍스 소재도 좋아서 인체에 무해할거 같아서 믿고 구매했어요^^-강아지가 젤 먼저 고른 장난감ㅋㅋㅋ 라텍스를 잘 좋아해용 굿-강아지 잘 가지고 놉니다  많이 파세요-삑삑이 소리나는 부분 납작한 곳이 있어서 구르다가 멈춰서 좋아요.-먼저 구매했었는데 좋아해서 재구매했어요.먼저구매한건 삑삑이가빠졌는데 새거오자마자 새거만  가지고노네요-삑삑ㅇㅣ 좋아해서 대량 구매해서 사용하고 있어요 \n소리도 잘나고 모양다틀려서 너무 귀여워요-공 말랑말랑해서 잘 튀겨지고 아이도 좋아해요 생각보다는 크기가 작아요 테니스공보다 더 작아요-저번에 사서 너무 잘 가지고 놀았는데 어디로 갔는지 안보여서 사료 사면서 같이 주문했어요-이 제품은 잘믄들어진 공 입니다 추천~-늘 여기서 구매하네요 추천합니다-애기가 갑자기 너무 커서 작아져서 놀진 않지만 몇일 잘 가지구 놀았어요-배송도 빠르고 좋습니당-가격만족 제품만족 배송만족합니다-5살 푸들인데 너무 좋아하네요 안 놀아도 계속 물고 있거나 꼭 주위에 놓고 있네욯ㅎ-삑삑이 생각보다 튼튼해서 아직 잘 사용하고 있어요-삑삑소리 잘나서 강아지가 좋아해요~-사실 삑빅이 인형 다 터트려서 속에 삑삑이 대신으로 사고 있는데 그냥도 잘 가

In [39]:
okt = Okt()
training_data = []
sentences = corpus.split("-")

for i in range(len(sentences)):
      sentences[i] = sentences[i].strip()
      words_and_tags = okt.pos(sentences[i])

      filtered_words = [word for word, tag in words_and_tags
                        if tag in ['Noun', 'Verb', 'Adjective']
                        and word not in stop_words]

      x = [word.strip(string.punctuation) for word in filtered_words]
      x = [word for word in filtered_words if len(x) > 1]
      training_data.append(x)

training_data

[['진짜', '귀얍', '잘', '갖고', '놀아요'],
 ['강아지', '최애', '공', '입니다', '노랑', '색', '좋아하네요'],
 ['노견',
  '푸',
  '들이',
  '놀이',
  '좋아하는데',
  '이빨',
  '약해서',
  '탄탄한',
  '공',
  '잘',
  '못',
  '물더라구요',
  '이',
  '공',
  '추천',
  '받아서',
  '써',
  '보니',
  '말랑하면서',
  '잘',
  '던져지고',
  '구성',
  '좋아서',
  '쓰다가',
  '새로',
  '구입',
  '하게',
  '됐어요',
  '개',
  '구매',
  '하니',
  '알아서',
  '색',
  '별로',
  '챙겨주셔서',
  '감사합니다'],
 ['저희', '강아지', '진짜', '좋아하는', '공이', '개', '샀는데', '좋아요'],
 ['것',
  '빨강이',
  '골',
  '줬는데',
  '환장하구',
  '놀아욤',
  '막',
  '뜯어',
  '버리는',
  '애도',
  '아니고',
  '짱짱해서',
  '가지',
  '놀',
  '것',
  '같아',
  '더',
  '좋네요'],
 ['가격', '만족', '배송', '만족', '제품', '만족'],
 ['아주', '잘', '쓰고', '있어요'],
 ['뽁뽁',
  '리나',
  '공',
  '좋아해서',
  '늘',
  '갖고',
  '놀던',
  '뽁뽁',
  '장난감',
  '소리',
  '안나',
  '가격',
  '저렴하고',
  '해서',
  '하나',
  '구매',
  '해봤어요',
  '던져주니',
  '아주',
  '좋아해서',
  '만족합니다',
  '라텍스',
  '소재',
  '좋아서',
  '인체',
  '무',
  '해',
  '할거',
  '같아서',
  '믿고',
  '구매',
  '했어요'],
 ['강아지', '젤', '먼저', '고른', '장난감', '라텍스', '잘', '좋아해용', '굿'],
 ['강아지', '잘', '가